## Imports

In [215]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [216]:
cwd = os.getcwd()

In [217]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [218]:
import financialstatements

## Config

In [219]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

In [220]:
GENERAL_LEDGER = os.path.join(cwd, config.get("data_outputs_directory", "GENERAL_LEDGER"))

In [221]:
TRIAL_BALANCE_INPUT = os.path.join(cwd, config.get("data_inputs_directory", "TRIAL_BALANCE_INPUT"))

In [222]:
TRIAL_BALANCE_OUTPUT = os.path.join(cwd, config.get("data_outputs_directory", "TRIAL_BALANCE_OUTPUT"))

In [223]:
gl_paths = financialstatements.creating_input_paths(GENERAL_LEDGER)

In [224]:
tb_paths = financialstatements.creating_input_paths(TRIAL_BALANCE_INPUT)

In [225]:
gl_df = pd.read_csv(gl_paths[0])

In [226]:
tb_df = pd.read_csv(tb_paths[0])

## ETL

1. General Ledger per month
2. Trial balance per month
3. Income Statement per month
4. Closing entries per month
5. Closing trial balance per month
6. Balance Sheet per month

In [227]:
gl_df['Date'] = pd.to_datetime(gl_df['Date'])

In [228]:
gl_df['Month_Num'] = gl_df['Date'].dt.month
gl_df['Month'] = gl_df['Date'].dt.strftime('%b').str.lower()

In [229]:
df = gl_df.groupby(['GL_Code', 'Account', 'Month_Num', 'Month'])[['DEBIT', 'CREDIT']].sum().reset_index()

In [230]:
tb_df = tb_df[['GL_Code', 'End_Balance']]

In [231]:
df = pd.merge(df, tb_df, how = 'left', on = 'GL_Code')

In [232]:
df = df.rename(columns = {'End_Balance':'Beg_Balance'})

In [233]:
df['Movement'] = df['DEBIT'] - df['CREDIT']

In [234]:
df['End_Balance'] = df['Beg_Balance'] + df['DEBIT'] - df['CREDIT']

In [235]:
df['End_Balance'] = df['End_Balance'].abs()

In [236]:
month_num = 1

In [237]:
type = 'unadjusted_trial_balance'

In [238]:
month = 'jan'

In [239]:
file_name = f"{month_num}_{type}_{month}.csv"

In [240]:
file_path = os.path.join(TRIAL_BALANCE_OUTPUT, file_name)

In [241]:
df.to_csv(file_path, index=False)